In [12]:

from detector import Detector
import glob
import matplotlib.pyplot as plt
import pandas as pd
import os
import json
from PIL import Image
from IPython.display import display
import random

test_text = '''
Do you have a problem that you are struggling to solve? Why don't you ask your friends for advice? When people ask for advice on solving a problem, often times they speak to more than one person. This is because different views are better for figuring out a tough problem, many opinions are better than one, and other people may have experienced a problem like yours and may be able to help you in making better decisions.
'''

# AI Text Image Recognition

Is there some similarity to Malware detection, where it's common to translate malware code to bytes and then images, which are then fed into NN/DL networks? Let's test a bit.

In [6]:
ai = json.load(open("E:\\Python_Projects\\PrismAI\\PrismAI\\src\\outputs\\generator_1_gpt2_100_outputs.json", encoding='utf8'))
human = json.load(open("E:\\Python_Projects\\PrismAI\\PrismAI\\src\\outputs\\generator_0_gpt2_100_outputs.json", encoding='utf8'))
print(ai[0]['metadata'])
print(human[0]['metadata'])

{'full_text': " In recent years, technology has had a profound impact on our daily lives and the world around us. From staying connected with loved ones to ordering food online through an app, technology has made our lives easier and more convenient. However, with great power comes great responsibility, and technology can also have negative consequences if used improperly.\n\nOne example of this is the spread of misinformation through the internet. It's easy to find articles and sources that may not be accurate or reliable, which can lead to confusion and even harm. Additionally, technology can also pose a threat to our privacy and security if we are not careful about the websites and apps we use.\n\nDespite these potential drawbacks, I believe that technology can be a powerful tool for achieving great things. For example, technology has revolutionized the way we learn and access information. With the internet, we can easily find resources and information on any topic, making it easier

## Do Function

In [125]:
def do(detector_outputs):
    for output in detector_outputs:
        meta = output['metadata']
        height = meta['sample_rate']
        width = meta['sample_sequence_length']
        results = output['ensemble_results'][0]['sample_results']
        
        image = Image.new('RGBA', (100, 20), color=(0,0,0,255))
        pixels = image.load()

        x = 1
        offset_x = x
        y = 1

        for result in results:
            steps = result['model_outputs']['steps']
            steps.sort(key= lambda x: x['step'])

            for step in steps:                
                r = 255
                g = 255
                b = 255
                
                all_probs = step['top_k_probs']
                all_probs.sort(reverse=True)
                other_probs = all_probs[1:]
                
                # mid
                pixels[x, y] = (r, g, b, int(step['target_prob'] * 255))
                # right
                pixels[x+1, y] = (r, g, b, int(other_probs[0] * 255))
                # left
                pixels[x-1, y] = (r, g, b, int(other_probs[1] * 255))
                # down
                pixels[x, y-1] = (r, g, b, int(other_probs[2] * 255))
                # up
                pixels[x, y+1] = (r, g, b, int(other_probs[3] * 255))
                
                x += 3
            
            offset_x = int((offset_x % 2 == 1))
            x = offset_x
            y += 3
        display(image)

                 
            
def draw_image(width, height):
    image = Image.new('RGBA', (width, height))
    pixels = image.load()
    # iterate over all pixels
    for y in range(height):
        for x in range(width):
            r = random.randint(0, 255)
            g = random.randint(0, 255)
            b = random.randint(0, 255)
            a = random.randint(0, 1)
            pixels[x, y] = (r, g, b, int(a * 255))  # Set the pixel RGBA values
    return image
    

In [127]:
do(human[:5])
do(ai[:5])
